## Applying Stochastic Methods
### Getting Started
This tutorial focuses on using stochastic methods to estimate ultimates. 

Note that a lot of the examples shown here might not be applicable in a real world scenario, and is only meant to demonstrate some of the functionalities included in the package. The user should always exercise their best actuarial judgement, and follow any applicable laws, the Code of Professional Conduct, and applicable Actuarial Standards of Practice.

Be sure to make sure your packages are updated. For more info on how to update your pakages, visit [Keeping Packages Updated](https://chainladder-python.readthedocs.io/en/latest/install.html#keeping-packages-updated).

In [ ]:
# Black linter, optional
%load_ext lab_black

import pandas as pd
import numpy as np
import chainladder as cl
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os

%matplotlib inline

print("pandas: " + pd.__version__)
print("numpy: " + np.__version__)
print("chainladder: " + cl.__version__)

### Intro to MackChainladder

Like the basic `Chainladder` method, the `MackChainladder` is entirely specified by its selected development pattern. In fact, it is the basic `Chainladder`, but with extra features.

In [ ]:
clrd = (
    cl.load_sample("clrd")
    .groupby("LOB")
    .sum()
    .loc["wkcomp", ["CumPaidLoss", "EarnedPremNet"]]
)

cl.Chainladder().fit(clrd["CumPaidLoss"]).ultimate_ == cl.MackChainladder().fit(
    clrd["CumPaidLoss"]
).ultimate_

Let's create a Mack's Chainladder model.

In [ ]:
mack = cl.MackChainladder().fit(clrd["CumPaidLoss"])

MackChainladder has the following additional fitted features that the deterministic `Chainladder` does not:

- `full_std_err_`:  The full standard error
- `total_process_risk_`: The total process error
- `total_parameter_risk_`: The total parameter error
- `mack_std_err_`: The total prediction error by origin period
- `total_mack_std_err_`: The total prediction error across all origin periods

Notice these are all measures of uncertainty, but where can they be applied? Let's start by examining the `link_ratios` underlying the triangle between age 12 and 24.

In [ ]:
clrd_first_lags = clrd[clrd.development <= 24][clrd.origin < "1997"]["CumPaidLoss"]
clrd_first_lags

A simple average link-ratio can be directly computed.

In [ ]:
clrd_first_lags.link_ratio.to_frame().mean()[0]

We can also verify that the result is the same as the `Development` object.

In [ ]:
cl.Development(average="simple").fit(clrd["CumPaidLoss"]).ldf_.to_frame().values[0, 0]

### The Linear Regression Framework

Mack noted that the estimate for the LDF is really just a linear regression fit. In the case of using the `simple` average, it is a weighted regression where the weight is $\left (\frac{1}{X}  \right )^{2}$.

Let's take a look at the fitted coefficient and verify that this ties to the direct calculations that we made earlier.
With the regression framework in hand, we can get more information about our LDF estimate than just the coefficient.

In [ ]:
y = clrd_first_lags.to_frame().values[:, 1]
x = clrd_first_lags.to_frame().values[:, 0]

model = sm.WLS(y, x, weights=(1 / x) ** 2)
results = model.fit()
results.summary()

By toggling the weights of our regression, we can handle the most common types of averaging used in picking loss development factors.
- For simple average, the weights are $\left (\frac{1}{X}  \right )^{2}$
- For volume-weighted average, the weights are $\left (\frac{1}{X}  \right )$
- For "regression" average, the weights are 1

In [ ]:
print("Simple average:")
print(
    round(
        cl.Development(average="simple")
        .fit(clrd_first_lags)
        .ldf_.to_frame()
        .values[0, 0],
        10,
    )
    == round(sm.WLS(y, x, weights=(1 / x) ** 2).fit().params[0], 10)
)

print("Volume-weighted average:")
print(
    round(
        cl.Development(average="volume")
        .fit(clrd_first_lags)
        .ldf_.to_frame()
        .values[0, 0],
        10,
    )
    == round(sm.WLS(y, x, weights=(1 / x)).fit().params[0], 10)
)

print("Regression average:")
print(
    round(
        cl.Development(average="regression")
        .fit(clrd_first_lags)
        .ldf_.to_frame()
        .values[0, 0],
        10,
    )
    == round(sm.OLS(y, x, weights=1).fit().params[0], 10)
)

The regression framework is what the `Development` estimator uses to set development patterns. Although we discard the information in the deterministic methods, in the stochastic methods, `Development` has two useful statistics for estimating reserve variability, both of which come from the regression framework. The stastics are `sigma_` and `std_err_` , and they are used by the `MackChainladder` estimator to determine the prediction error of our reserves.

In [ ]:
dev = cl.Development(average="simple").fit(clrd["CumPaidLoss"])

In [ ]:
dev.sigma_

In [ ]:
dev.std_err_

Remember that `std_err_` is calculated as $\frac{\sigma}{\sqrt{N}}$.

In [ ]:
np.round(
    dev.sigma_.to_frame().transpose()["(All)"].values
    / np.sqrt(clrd["CumPaidLoss"].age_to_age.to_frame().count()).values,
    4,
)

Since the regression framework uses the weighting method, we can easily turn "on and off" any observation we want using the dropping capabilities such as `drop_valuation` in the `Development` estimator. Dropping link ratios not only affects the `ldf_` and `cdf_`, but also the `std_err_` and `sigma` of the estimates.

Can we eliminate the 1988 valuation from our triangle, which is identical to eliminating the first observation from our 12-24 regression fit? Let's calculate the `std_err` for the `ldf_` of ages 12-24, and compare it to the value calculated using the weighted least squares regression.

In [ ]:
clrd["CumPaidLoss"]

In [ ]:
round(
    cl.Development(average="volume", drop_valuation="1988")
    .fit(clrd["CumPaidLoss"])
    .std_err_.to_frame()
    .values[0, 0],
    8,
) == round(sm.WLS(y[1:], x[1:], weights=(1 / x[1:])).fit().bse[0], 8)

With `sigma_` and `std_err_` in hand, Mack goes on to develop recursive formulas to estimate `parameter_risk_` and `process_risk_`.

In [ ]:
mack.parameter_risk_

In [ ]:
mack.process_risk_

### Assumption of Independence
The Mack model makes a lot of assumptions about independence (i.e. the covariance between random processes is 0). This means that many of the Variance estimates in the `MackChainladder` model follow the form of $Var(A+B) = Var(A)+Var(B)$. 

First, `mack_std_err_`<sup>2</sup> $=$ `parameter_risk_`<sup>2</sup> $+$ `process_risk_`<sup>2</sup>, the parameter risk and process risk is assumed to be independent. 

In [ ]:
mack.parameter_risk_ ** 2 + mack.process_risk_ ** 2

In [ ]:
mack.mack_std_err_ ** 2

Second, `total_process_risk_` <sup>2</sup> $= \sum_{origin} $ `process_risk_` <sup>2</sup>, the process risk is assumed to be independent between origins.

In [ ]:
mack.total_process_risk_ ** 2

In [ ]:
(mack.process_risk_ ** 2).sum(axis="origin")

Lastly, independence is also assumed to apply to the overall standard error of reserves, as expected.

In [ ]:
(mack.parameter_risk_ ** 2 + mack.process_risk_ ** 2).sum(axis=2).sum(axis=3)

In [ ]:
(mack.mack_std_err_ ** 2).sum(axis=2).sum(axis=3)

This over-reliance on independence is one of the weaknesses of the `MackChainladder` method. Nevertheless, if the data align with this assumption, then `total_mack_std_err_` is a reasonable esimator of reserve variability.

### Mack Reserve Variability
The `mack_std_err_` at ultimate is the reserve variability for each `origin` period.

In [ ]:
mack.mack_std_err_[
    mack.mack_std_err_.development == mack.mack_std_err_.development.max()
]

With the `summary_` method, we can more easily look at the result of the `MackChainladder` model.

In [ ]:
mack.summary_

Let's visualize the paid to date, the estimated reserves, and their standard errors with a histogram.

In [ ]:
plt.bar(
    mack.summary_.to_frame().index.year,
    mack.summary_.to_frame()["Latest"],
    label="Paid",
)
plt.bar(
    mack.summary_.to_frame().index.year,
    mack.summary_.to_frame()["IBNR"],
    bottom=mack.summary_.to_frame()["Latest"],
    yerr=mack.summary_.to_frame()["Mack Std Err"],
    label="Reserves",
)
plt.legend(loc="upper left")
plt.ylim(0, 1800000)

We can also simulate the (assumed) normally distributed IBNR with `np.random.normal()`.

In [ ]:
ibnr_mean = mack.ibnr_.sum()
ibnr_sd = mack.total_mack_std_err_.values[0, 0]
n_trials = 10000

np.random.seed(2021)
dist = np.random.normal(ibnr_mean, ibnr_sd, size=n_trials)

plt.hist(dist, bins=50)

### ODP Bootstrap Model
The `MackChainladder` focuses on a regression framework for determining the variability of reserve estimates. An alternative approach is to use the statistical bootstrapping, or sampling from a triangle with replacement to simulate new triangles.

Bootstrapping imposes less model constraints than the `MackChainladder`, which allows for greater applicability in different scenarios. Sampling new triangles can be accomplished through the `BootstrapODPSample` estimator. This estimator will take a single triangle and simulate new ones from it. To simulate new triangles randomly from an existing triangle, we specify `n_sims` with how many triangles we want to simulate, and access the `resampled_triangles_` attribute to get the simulated triangles. Notice that the shape of `resampled_triangles_` matches `n_sims` at the first index.

In [ ]:
samples = (
    cl.BootstrapODPSample(n_sims=10000).fit(clrd["CumPaidLoss"]).resampled_triangles_
)
samples

Alternatively, we could use `BootstrapODPSample` to transform our triangle into a resampled set.

The notion of the ODP Bootstrap is that as our simulations approach infinity, we should expect our mean simulation to converge on the basic `Chainladder` estimate of of reserves.

Let's apply the basic chainladder to our original triangle and also to our simulated triangles to see whether this holds true.

In [ ]:
ibnr_cl = cl.Chainladder().fit(clrd["CumPaidLoss"]).ibnr_.sum()
ibnr_bootstrap = cl.Chainladder().fit(samples).ibnr_.sum("origin").mean()

print(
    "Chainladder's IBNR estimate:", ibnr_cl,
)
print(
    "BootstrapODPSample's mean IBNR estimate:", ibnr_bootstrap,
)
print("Difference $:", ibnr_cl - ibnr_bootstrap)
print("Difference %:", abs(ibnr_cl - ibnr_bootstrap) / ibnr_cl)

### Using Deterministic Methods with Bootstrapped Samples
Our `samples` is just another triangle object with all the functionality of a regular triangle.  This means we can apply any functionality we want to our `samples` including any deterministic methods we learned about previously.

In [ ]:
pipe = cl.Pipeline(
    [("dev", cl.Development(average="simple")), ("tail", cl.TailConstant(1.05))]
)
pipe.fit(samples)

Now instead of a single `ldf_` (and `cdf_`) array across developmental ages, we have 10,000 arrays of `ldf_` (and `cdf_`).

In [ ]:
pipe.named_steps.dev.ldf_.iloc[0]

In [ ]:
pipe.named_steps.dev.ldf_.iloc[1]

In [ ]:
pipe.named_steps.dev.ldf_.iloc[9999]

This allows us to look at the varibility of any fitted property used. Let's look at the distribution of the age-to-age factor between age 12 and 24, and compare it against the LDF calculated from the non-bootstrapped triangle.

In [ ]:
resampled_ldf = pipe.named_steps.dev.ldf_
plt.hist(pipe.named_steps.dev.ldf_.values[:, 0, 0, 0], bins=50)

orig_dev = cl.Development(average="simple").fit(clrd["CumPaidLoss"])
plt.axvline(orig_dev.ldf_.values[0, 0, 0, 0], color="black", linestyle="dashed")

### Bootstrap vs Mack
We can approximate some of the Mack's parameters calculated using the regression framework.

In [ ]:
bootstrap_vs_mack = resampled_ldf.std("index").to_frame().T
bootstrap_vs_mack.rename(columns={"(All)": "Std_Bootstrap"}, inplace=True)
bootstrap_vs_mack = bootstrap_vs_mack.merge(
    orig_dev.std_err_.to_frame().T, left_index=True, right_index=True
)
bootstrap_vs_mack.rename(columns={"(All)": "Std_Mack"}, inplace=True)

bootstrap_vs_mack

In [ ]:
width = 0.3
ages = np.arange(len(bootstrap_vs_mack))

plt.bar(
    ages - width / 2,
    bootstrap_vs_mack["Std_Bootstrap"],
    width=width,
    label="Bootstrap",
)
plt.bar(ages + width / 2, bootstrap_vs_mack["Std_Mack"], width=width, label="Mack")
plt.legend(loc="upper right")
plt.xticks(ages, bootstrap_vs_mack.index)

While the `MackChainladder` produces statistics about the mean and variance of reserve estimates, the variance or precentile of reserves would need to be fited using maximum likelihood estimation or method of moments. However, for `BootstrapODPSample` based fits, we can use the empirical distribution from the samples directly to get information about variance or the precentile of the IBNR reserves.

In [ ]:
ibnr = cl.Chainladder().fit(samples).ibnr_.sum("origin")

ibnr_std = ibnr.std()
print("Standard deviation of reserve estimate: " + f"{round(ibnr_std,0):,}")
ibnr_99 = ibnr.quantile(q=0.99)
print("99th percentile of reserve estimate: " + f"{round(ibnr_99,0):,}")

Let's see how the `MackChainladder` reserve distribution compares to the `BootstrapODPSample` reserve distribution.

In [ ]:
plt.hist(ibnr.to_frame(), bins=50, label="Bootstrap", alpha=0.3)
plt.hist(dist, bins=50, label="Mack", alpha=0.3)
plt.legend(loc="upper right")

### Bootstrapping with the Bornhuetter-Ferguson Method

So far, we've only applied the multiplicative methods (i.e. basic chainladder) in a stochastic context. It is possible to use an expected loss method like the `BornhuetterFerguson` method does. 

To do this, we will need an exposure vector.

In [ ]:
clrd["EarnedPremNet"].latest_diagonal

Passing an `apriori_sigma` to the `BornhuetterFerguson` estimator tells it to consider the `apriori` selection itself as a random variable. Fitting a stochastic `BornhuetterFerguson` looks very much like the determinsitic version. Let's assume that the `apriori` is 80% (of `clrd["EarnedPremNet"]`) and its standard deviation is 10%.

In [ ]:
bf = cl.BornhuetterFerguson(apriori=0.80, apriori_sigma=0.10)
bf.fit(samples, sample_weight=clrd["EarnedPremNet"].latest_diagonal)

Let's restate or sampled triangles so that the upper left portion of the triangle with known values. We will need to start with the `full_triangle_`, then take out the upper left (the simulated values) with `X_`, then add back the actual values from the raw triangle.

In [ ]:
restated_triangle = bf.full_triangle_ - bf.X_ + clrd["CumPaidLoss"]

We can also look at how a certain origin period developed with the sampled triangles. Let's take a look at origin year 1995.

In [ ]:
restated_triangle_1995_df = restated_triangle[
    restated_triangle.origin == "1995"
].to_frame()
restated_triangle_1995_df

For simplicity, let's only graph the first 1,000 simulations. As expected, plotting the expected development of our full triangle over time from the Bootstrap `BornhuetterFerguson` model fans out to greater uncertainty the farther we get from our valuation date. And notice that for 1997 and prior (age 36 and prior), there is no variability as we have restated the simulated triangles with actual data.

In [ ]:
plt.plot(
    restated_triangle_1995_df.T.reset_index(drop=True).iloc[:, 0:1000],
    color="blue",
    alpha=0.01,
);
plt.xticks(
    np.arange(0, 12, 1),
    ["12", "24", "36", "48", "60", "72", "84", "96", "108", "120", "132", "Ult"],
);

### Recap
- The Mack method approaches stochastic reserving from a regression point of view
- Bootstrap methods approach stochastic reserving from a simulation point of view
- When the assumptions of the model are not violated, they will both produce resonably consistent estimates of reserve variability
- Mack does impose more assumptions (i.e. constraints) on the reserve estimate making the Bootstrap approach more suitable in a broader set of applciations
- Both methods converge to their corresponding deterministic point estimates